# Find Days To Use for Lineup Optimization
This notebook has code that will list statistics for games days for a season of a sport. Use this to figure out which days to use as test cases for optimizing lineup generation settings.

1. Find days to run analysis on. Pick date(s) that have a desired number of games and players.

In [ ]:
from IPython import display
import os
import ipywidgets

from fantasy_py.util.notebook import DBFileDropdown, SeasonSelector

output = ipywidgets.widgets.Output()
db_dropdown = DBFileDropdown(path_to_dbs=os.environ['FANTASY_HOME'])
display.display(db_dropdown)
season_sel = SeasonSelector(db_dropdown, output_widget=output)
display.display(season_sel)
display.display(output)

In [ ]:
from sqlalchemy import func
import qgrid
from pandas import DataFrame

from fantasy_py import db

seasons = (2016, 2017, 2018)

os.chdir(os.environ['FANTASY_HOME'])
db_obj = db.get_db_obj(db_dropdown.value)

data = {'date': [],
        'games': []}

with db_obj.session_scoped() as session:
    for game_date, count in session.query(db.Game.date, func.count()) \
                                   .filter(db.Game.season.in_(seasons)) \
                                   .group_by(db.Game.date) \
                                   .order_by(db.Game.date) \
                                   .all():
        data['date'].append(game_date)
        data['games'].append(count)

    
df = DataFrame.from_dict(data)
# note that the number of players per game is uniquely defined by sport
if db_obj.db_manager.ABBR == 'mlb':
    df['players'] = df['games'] * 9
else:
    raise NotImplemented("do something for this sport's player count")

display.display(qgrid.QgridWidget(df=df))